In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sequilt import Sequilt
from sequilt.model import EventGraph, Sequlet, LabelModel

In [3]:
from datasets import load_dataset
from sequilt.data import get_ids, get_tokenizer

ds = load_dataset("neuralbioinfo/bacterial_promoters")
tokenizer = get_tokenizer(type="dna", k=1)
ids, tokens = get_ids(
  ds["test_multispecies"]["segment"], tokenizer, max_tokens=32
)
labels = [
  LabelModel(value=value, name=name)
  for value, name in tokenizer._id_to_token.items()
  if value != 0
]

labels = sorted(labels, key=lambda x: x.name)

100%|██████████| 22582/22582 [00:01<00:00, 16359.11it/s]


In [4]:
# Language Dataset
# from datasets import load_dataset
# from sequilt.data import get_ids, get_tokenizer, get_featured_ids

# ds = load_dataset("ajaykarthick/imdb-movie-reviews")
# tokenizer = get_tokenizer(type="language")
# ids, tokens = get_ids(
#   ds['test']["review"], tokenizer, max_tokens=32
# )
# featured_ids = get_featured_ids(ids, tokenizer, "tf-idf", n_features=20)
# feature_mask = np.isin(ids, featured_ids)
# ids = np.where(feature_mask, ids, 0)
# labels = [
#   LabelModel(value=id, name=tokenizer.id_to_token(id))
#   for id in featured_ids
# ]

In [5]:
G = EventGraph(ids)
sequilt = Sequilt(sequence_length=ids.shape[1], n_sequences=ids.shape[0], labels=labels)
sequilt

Sequilt(labels=[{'value': 3, 'name': 'A'}, {'value': 1, 'name': 'C'}, {'value': 2, 'name': 'G'}, {'value': 4, …

In [6]:
# Draw edges
from time import sleep
G = EventGraph(ids)
sequilt.sequlets = []

for id, (event1, event2, cooccurence) in enumerate(G.sorted_edges):
  if cooccurence < 5:
    break
  sequlet = Sequlet(id, [event1, event2])
  sleep(0.15)
  sequilt.draw_sequlet(sequlet)
  G.remove_events_from([event1, event2])
  
# Draw nodes
for event in G.events:
  sequlet = Sequlet(len(sequilt), [event])
  sequilt.draw_sequlet(sequlet)
  sleep(0.15)
  # G.remove_event(event)